In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [3]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [5]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
# %matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.layers.advanced_activations import ReLU
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
import random
import math
import matplotlib.image as mpimg
import cv2
import os

In [7]:
IMG_WIDTH=216
IMG_HEIGHT=216
path='augmented_dataset'

In [8]:
def create_dataset(path):
  img_data_array = []
  for img in os.listdir(path):
    
    image= cv2.imread( os.path.join(path,img), cv2.COLOR_BGR2RGB)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    img_data_array.append(image)


  return img_data_array
# extract the image array and class name
img_data=create_dataset('augmented_dataset')

In [9]:
train_data = np.asarray(img_data)
train_data.shape

(928, 216, 216, 3)

In [14]:
target_path = 'target_image'

In [15]:
target_data=create_dataset('target_image')

In [16]:
target_data = np.asarray(target_data)
target_data.shape

(928, 216, 216, 3)

In [ ]:
# def load_dataset():
#   # train_data = pd.read_csv('train_aug_dataset.csv')
#   y_train = pd.read_csv('augmented_labels_delta.csv')
#   target_data = pd.read_csv('target_data.csv')
#   # target_label = pd.read_csv('target_label.csv')
#   return train_data, y_train, target_data
# train_data, y_train, target_data = load_dataset()

In [17]:
y_train = pd.read_csv('augmented_labels_delta.csv')

In [18]:
num_classes = 4

In [ ]:
# train_data = np.array(train_data.iloc[:,:])
# train_data = np.array([np.reshape(i,(128,128)) for i in train_data])
# target_data = np.array(target_data.iloc[:,:])
# target_data = np.array([np.reshape(i,(128,128)) for i in target_data])

In [19]:
train_data.shape
target_data.shape

(928, 216, 216, 3)

In [20]:
# train_data = train_data.astype('float32')
# train_data = train_data/255

# rgb_batch = np.repeat(train_data[..., np.newaxis], 3, -1)
# rgb_batch.shape

# target_data = target_data.astype('float32')
# target_data = target_data/255

# rgb_target = np.repeat(target_data[..., np.newaxis], 3, -1)
# rgb_target.shape

In [21]:
# target_data = rgb_target

In [22]:
# target_data.shape

In [23]:
x_train, x_test, y_train, y_test = train_test_split(train_data, y_train, test_size = 0.3, random_state=666)
target_xtrain, target_xtest = train_test_split(target_data, test_size = 0.3, random_state=666)

In [24]:
input_shape = (216, 216, 3)

In [ ]:
# print(y_test)

In [31]:
def VGG_16(input_image):
    model = Conv2D(64, (3,3), padding="same", input_shape = input_shape)(input_image)
    model = ReLU(max_value=1.0)(model)
    model = Conv2D(64, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)

    model = Conv2D(128, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    # model = Conv2D(128, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    # model = Dropout(0.25)(model)

    # model = Conv2D(256, (3,3), padding="same")(model)
    # model = ReLU(max_value=1.0)(model)
    model = Conv2D(256, (3,3), padding="same")(model)
    model = ReLU(max_value=1.0)(model)
    model = MaxPooling2D(pool_size=(2,2), padding="same")(model)
    model = Dropout(0.25)(model)


    return model

In [32]:
current_input = Input(shape=input_shape)
current_model = VGG_16(current_input)

target_input = Input(shape=input_shape)
target_model = VGG_16(target_input)

In [33]:
conv = concatenate([current_model, target_model])

In [34]:
conv = Flatten()(conv)
dense = Dense(512)(conv)
dense = ReLU(max_value=None)(dense)
dense = Dropout(0.5)(dense)
# dense = Dense(512)(dense)
# dense = ReLU(max_value=1.0)(dense)
# dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='linear')(dense)
# output = math.floor(output)
model = Model(inputs=[current_input, target_input], outputs=[output])

opt = Adam(learning_rate=0.0001)

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])


In [35]:
best_weights_file="weights.aug.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_mse', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

In [36]:
model.fit([x_train, target_xtrain], y_train,
          epochs=100,
          validation_split=0.3,
          callbacks = callbacks,
          verbose=1)

Epoch 1/100
15/15 [==============================] - 13s 678ms/step - loss: 192.7166 - mse: 192.7166 - val_loss: 184.1483 - val_mse: 184.1483

Epoch 00001: val_mse improved from -inf to 184.14833, saving model to weights.aug.best.hdf5
Epoch 2/100
15/15 [==============================] - 9s 570ms/step - loss: 188.3797 - mse: 188.3797 - val_loss: 193.1387 - val_mse: 193.1387

Epoch 00002: val_mse improved from 184.14833 to 193.13866, saving model to weights.aug.best.hdf5
Epoch 3/100
15/15 [==============================] - 9s 575ms/step - loss: 176.6090 - mse: 176.6090 - val_loss: 190.8761 - val_mse: 190.8761

Epoch 00003: val_mse did not improve from 193.13866
Epoch 4/100
15/15 [==============================] - 9s 574ms/step - loss: 166.9187 - mse: 166.9187 - val_loss: 188.7865 - val_mse: 188.7865

Epoch 00004: val_mse did not improve from 193.13866
Epoch 5/100
15/15 [==============================] - 9s 578ms/step - loss: 167.9042 - mse: 167.9042 - val_loss: 184.7536 - val_mse: 184.75

KeyboardInterrupt: ignored

In [ ]:
model.evaluate([x_test, target_xtest], y_test)

9/9 [==============================] - 1s 135ms/step - loss: 115.3931 - mse: 115.3931


[115.3930892944336, 115.3930892944336]

In [ ]:
prediction = model.predict([x_test, target_xtest])
prediction = np.rint(prediction)

In [ ]:
print(prediction)

[[ 29.  -8.  -1.   6.]
 [ -0.  -0.  -0.   0.]
 [ -0.  -0.  -0.   0.]
 ...
 [-10.  -3.  -5.   6.]
 [ -0.  -0.  -0.   0.]
 [  1.   3.  -5.   1.]]


In [ ]:
np.savetxt('predictions.csv', prediction, delimiter=",")

In [ ]:
y_test.to_numpy()

array([[ 35, -13,   0,   7],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       ...,
       [-25, -18,   0,  -3],
       [  0,   0,   0,   0],
       [ 25, -18,   0,   2]])

In [ ]:
np.savetxt('ground_truth.csv', y_test, delimiter=',')

In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# img1 = cv2.imread('data65_27.png')
# img2 = cv2.imread('target_image_1.png')
#mg1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img1 = cv2.resize(img1, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img1)
# img1.shape

# img1 = np.expand_dims(img1, axis = 0)
# img1 = np.expand_dims(img1, axis = 3)
# img1 = img1.astype('float32')
# img1 = img1/255
# plt.imshow(img1.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
# from PIL import Image
# # Open the image form working directory
# img1 = Image.open('/content/data65_27.png')
# img2 = Image.open('/content/target_image_1.png')

In [ ]:
# from matplotlib import image
# from matplotlib import pyplot
# # load image as pixel array
# img1 = image.imread('/content/data65_27.png')
# img2 = image.imread('/content/target_image_1.png')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('test8_res_res_res_res_res_res_res_res_000011.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img= cv2.resize(img, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img.shape

img = np.reshape(img, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img = img/255
img = np.repeat(img[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img.shape

(1, 128, 128, 3)

In [ ]:
img2 = cv2.imread('target_image.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2= cv2.resize(img2, dsize=(128,128), interpolation = cv2.INTER_CUBIC)
# plt.imshow(img)
img2.shape

img2 = np.reshape(img2, (1, 128, 128))
# img = np.expand_dims(img, axis = 0)
# img = np.expand_dims(img, axis = 3)
# img = np.reshape(128,128)
# img = img.astype('float32')
img2 = img2/255
img2 = np.repeat(img2[..., np.newaxis], 3, -1)
# plt.imshow(img.reshape(128,128), cmap = plt.cm.binary)

In [ ]:
img2.shape

(1, 128, 128, 3)

In [ ]:
predictionimg = model.predict([img,img2])
predictionimg = np.rint(predictionimg)

In [ ]:
predictionimg

array([[-17.,  -3.,  -5.,   3.]], dtype=float32)